<a href="https://colab.research.google.com/github/summermccune/Tokenization-Testing-for-Malware-Data/blob/main/Word2Vec/Word2Vec%26SVM_trigrams.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from gensim.models import Word2Vec
import pandas as pd
import numpy as np
from gensim.utils import simple_preprocess
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import glob
import nltk
from nltk.tokenize import word_tokenize
from nltk.util import trigrams
nltk.download('punkt')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def read_files(malwarefolder, malwareType):
  for sample in malwarefolder:
    with open(sample, 'r') as f:
      data = f.read().split()
      #make opcodes into trigrams
      trigrms = list(trigrams(data))
      #make the bigrams each their own string
      trigrms = [f"{first} {second} {third}" for first, second, third in trigrms]
      data = trigrms
    #append sample to opcodes list in df
    df.loc[len(df)] = [data, malwareType]

In [ ]:
#creating dataframe for samples and label
df = pd.DataFrame(columns = ['opcodes','label'])

#specify paths for each malware
winwebsec = glob.glob("/content/drive/MyDrive/Data/malware2/winwebsec/*.txt")
zbot = glob.glob("/content/drive/MyDrive/Data/malware2/zbot/*.txt")
zeroaccess = glob.glob("/content/drive/MyDrive/Data/malware2/zeroaccess/*.txt")

#read files
read_files(winwebsec, 0)
read_files(zbot, 1)
read_files(zeroaccess, 2)

print(df.shape)
df.head()


(7801, 2)


opcodes  label
0  [push mov sub, mov sub push, sub push push, pu...      0
1  [push mov sub, mov sub mov, sub mov mov, mov m...      0
2  [push mov sub, mov sub xor, sub xor mov, xor m...      0
3  [push mov sub, mov sub push, sub push push, pu...      0
4  [mov push mov, push mov sub, mov sub lea, sub ...      0

In [ ]:
#word2vec
model = Word2Vec(df['opcodes'], min_count=1, vector_size=100)

In [ ]:
#SVM
X = np.array([np.mean([model.wv[word] for word in text if word in model.wv]
                        , axis=0) for text in df['opcodes']])
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
svm = SVC(kernel='rbf')
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)

#accuracy and conf matrix
print("Accuracy:", np.mean(y_pred == y_test))
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.9859154929577465
[[440   2   0]
 [  2 192   6]
 [  1   0 138]]
